In [1]:
from keras.applications import ResNet50
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from sklearn.preprocessing import LabelEncoder
from imutils import paths
import numpy as np
import argparse
import random
import os

Using TensorFlow backend.


In [2]:
from hdf5datasetwriter import HDF5DatasetWriter

In [3]:
import h5py

In [4]:
imagePaths = list(paths.list_images("/home/divyanshu/Desktop/Kaggle/CatsvsDogs/test1"))
labels = [p.split(os.path.sep)[-1].split(".")[0] for p in imagePaths]
print(imagePaths[2])
print(type(labels[0]))

/home/divyanshu/Desktop/Kaggle/CatsvsDogs/test1/1768.jpg
<class 'str'>


In [6]:
dataset = HDF5DatasetWriter((len(imagePaths), 2048), "/home/divyanshu/Desktop/Kaggle/CatsvsDogs/outputstest", dataKey = "features")

In [7]:
print(len(imagePaths))

12500


In [8]:
model = ResNet50(weights = "imagenet", include_top = False)

In [9]:
for i in np.arange(0, len(imagePaths), 32):
    batchPaths = imagePaths[i: i + 32]
    batchLabels = labels[i: i + 32]
    batchImages = []
    for (j, imagePath) in enumerate(batchPaths):
        image = load_img(imagePath, target_size = (224, 224))
        image = img_to_array(image)
        image = np.expand_dims(image, axis = 0)
        image = imagenet_utils.preprocess_input(image)
        batchImages.append(image)
    batchImages = np.vstack(batchImages)
    features = model.predict(batchImages, batch_size = 32)
    features = features.reshape((features.shape[0], 2048))
    dataset.add(features, batchLabels)
    print(i)
dataset.close()

0
32
64
96
128
160
192
224
256
288
320
352
384
416
448
480
512
544
576
608
640
672
704
736
768
800
832
864
896
928
960
992
1024
1056
1088
1120
1152
1184
1216
1248
1280
1312
1344
1376
1408
1440
1472
1504
1536
1568
1600
1632
1664
1696
1728
1760
1792
1824
1856
1888
1920
1952
1984
2016
2048
2080
2112
2144
2176
2208
2240
2272
2304
2336
2368
2400
2432
2464
2496
2528
2560
2592
2624
2656
2688
2720
2752
2784
2816
2848
2880
2912
2944
2976
3008
3040
3072
3104
3136
3168
3200
3232
3264
3296
3328
3360
3392
3424
3456
3488
3520
3552
3584
3616
3648
3680
3712
3744
3776
3808
3840
3872
3904
3936
3968
4000
4032
4064
4096
4128
4160
4192
4224
4256
4288
4320
4352
4384
4416
4448
4480
4512
4544
4576
4608
4640
4672
4704
4736
4768
4800
4832
4864
4896
4928
4960
4992
5024
5056
5088
5120
5152
5184
5216
5248
5280
5312
5344
5376
5408
5440
5472
5504
5536
5568
5600
5632
5664
5696
5728
5760
5792
5824
5856
5888
5920
5952
5984
6016
6048
6080
6112
6144
6176
6208
6240
6272
6304
6336
6368
6400
6432
6464
6496
6528
6560
6592
66

In [10]:
#Logistic Regression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression

In [14]:
import pickle
db = h5py.File("/home/divyanshu/Desktop/Kaggle/CatsvsDogs/outputstest", "r")
f = open("/home/divyanshu/Desktop/Kaggle/CatsvsDogs/trained_model/dogsvscats.pickle", "rb")
model = pickle.load(open("/home/divyanshu/Desktop/Kaggle/CatsvsDogs/trained_model/dogsvscats.pickle", "rb"))
p = model.predict_proba(db["features"])
print(p[0])
f.close()
#db.close()

[0.00115438 0.99884562]


In [79]:
def getExpandedScientificNotation(flt):
    was_neg = False
    if not ("e" in flt):
        return flt
    if flt.startswith('-'):
        flt = flt[1:]
        was_neg = True 
    str_vals = str(flt).split('e')
    coef = float(str_vals[0])
    exp = int(str_vals[1])
    return_val = ''
    if int(exp) > 0:
        return_val += str(coef).replace('.', '')
        return_val += ''.join(['0' for _ in range(0, abs(exp - len(str(coef).split('.')[1])))])
    elif int(exp) < 0:
        return_val += '0.'
        return_val += ''.join(['0' for _ in range(0, abs(exp) - 1)])
        return_val += str(coef).replace('.', '')
    if was_neg:
        return_val='-'+return_val
    return return_val

In [80]:
print(p[3][1])
print((db["labels"].shape))
print(db["labels"][3])
ls = [[db["labels"][i], getExpandedScientificNotation(str(p[i][1]))] for i in range(len(p))]
#ls = [[db["labels"][i], float(p[i][1])] for i in range(len(p))]
def getEl(elem):
    return int(elem[0])
ls.sort(key = getEl)

0.999616872143254
(12500,)
12264


In [83]:
print(ls[9])

['10', '0.001372449651015032']


In [84]:
import csv

In [87]:
with open('catsvsdogstest1.csv', 'w') as catsvsdogs:
    writer = csv.writer(catsvsdogs)
    writer.writerows(ls)
catsvsdogs.close()